In [1]:
from crewai import Agent,Task,Crew,Process,LLM

In [2]:
import agentops 
import os
from tavily import TavilyClient

In [11]:
os.environ["AGENTOPS_API_KEY"]= ""
os.environ["OPENAI_API_KEY"]= ""

In [12]:
agentops.init(
    api_key="",
    skip_auto_end_session=True
)

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=af37060f-236c-4cb1-8300-af4cacb6f2fa


In [13]:
output_dir="./ai-agent-output"
os.makedirs(output_dir,exist_ok=True)

basic_llm=LLM(model="gpt-4",temperature=0)
search_client=TavilyClient(api_key="")
scrape_client=Client(api_key="")

## Designing first agent:
To provide a list of suggested searched queries to be passed to the search engine 

In [14]:
from pydantic import BaseModel,Field
from typing import List

In [15]:
no_keywords=10

class SuggestedSearchQueries(BaseModel):
    queries:List[str]=Field(...,title="Suggested search queries to be passed the search engine.",min_items=1,max_items=no_keywords)
    

search_queries_recommendation_agent=Agent(
    role="Search Queries Recommendation Agent",
    goal="\n".join([ 
                "To provide a list of suggested search queries to be passed to the search engine.",
                "The queries must be varied and looking for specific items."
            ]),   
    backstory="The agent is desgined to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided.",
    llm=basic_llm,
    verbose=True,
)
search_queries_recommendation_task=Task(
    description="\n".join([
        "Ranyx is looking to buy {product_name} at the best prices (value for price strategy).",
        "The company target any of these websites to buy from:{websites_list}",
        "The company wants to reach all available products on the internet to be compared later in another stage.",
        "The stores must sell the product in {country_name}",
        "Generate at maximum {no_keywords} queries",
        "The search keywords must be in {language} language"
        "The search query must reach an ecommerce webpage for product, and not just a blog page."
    ]),
    expected_output="A JSON object containing a list of suggested search queries",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir,"step_1_suggested_search_queries.json"),
    agent=search_queries_recommendation_agent
)

## Run the AI Crew

In [16]:
ranyx_crew=Crew(
    agents=[
        search_queries_recommendation_agent,
    ],
    tasks=[
        search_queries_recommendation_task
    ],
    process=Process.sequential
)

In [17]:
crew_results=ranyx_crew.kickoff(
    inputs={
        "product_name":"coffee machine for the office",
        "websites_list":["www.amazon.eg","www.jumia.com.eg","www.noon.com/egypt-en"],
        "country_name":"Egypt",
        "no_keywords":10,
        "language":"Arabic"
    }
)

# Agent: Search Queries Recommendation Agent
## Task: Ranyx is looking to buy coffee machine for the office at the best prices (value for price strategy).
The company target any of these websites to buy from:['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/egypt-en']
The company wants to reach all available products on the internet to be compared later in another stage.
The stores must sell the product in Egypt
Generate at maximum 10 queries
The search query must reach an ecommerce webpage for product, and not just a blog page.


ERROR:root:LiteLLM call failed: litellm.NotFoundError: OpenAIException - Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Search Queries Recommendation Agent
## Task: Ranyx is looking to buy coffee machine for the office at the best prices (value for price strategy).
The company target any of these websites to buy from:['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/egypt-en']
The company wants to reach all available products on the internet to be compared later in another stage.
The stores must sell the product in Egypt
Generate at maximum 10 queries
The search query must reach an ecommerce webpage for product, and not just a blog page.


ERROR:root:LiteLLM call failed: litellm.NotFoundError: OpenAIException - Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Search Queries Recommendation Agent
## Task: Ranyx is looking to buy coffee machine for the office at the best prices (value for price strategy).
The company target any of these websites to buy from:['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/egypt-en']
The company wants to reach all available products on the internet to be compared later in another stage.
The stores must sell the product in Egypt
Generate at maximum 10 queries
The search query must reach an ecommerce webpage for product, and not just a blog page.


ERROR:root:LiteLLM call failed: litellm.NotFoundError: OpenAIException - Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



NotFoundError: litellm.NotFoundError: OpenAIException - Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [19]:
search_client.search("coffee machine site:amazon.eg")

## Agent B
to search google for products based on the suggested search queries

In [ ]:
class SingleSearchResult(BaseModel):
    title:str,
    url:str = Field(...,title="the page url")
    content:str,
    score:float,
    search_query:str
    
class AllSearchResults(BaseModel):
    results:List[SingleSearchResult]

In [ ]:
from crewai.tools import tool

@tool
def search_engine_tool(query:str):
    """Useful to find current information about any query related pages using a search engine"""
    return search_client.search(query)


search_engine_agent=Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is desgined to help in looking for products by searching for products based on the suggested search queries.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)
search_engine_task=Task(
    description="\n".join([
        "The Task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries",
        "ignore any susbicious links or not an ecommerce single product website link",
        "ignore any search results with confidence score less than ({score_th})",
        "The search product will be used to compare prices of product from different websites."
    ]),
    expected_outputs="A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir,"step_1_suggested_search_queries.json"),
    agent=search_engine_agent
)

In [ ]:
ranyx_crew=Crew(
    agents=[
        search_queries_recommendation_agent,
        search_engine_agent
    ],
    tasks=[
        search_queries_recommendation_task,
        search_engine_task
    ],
    process=Process.sequential
)

In [ ]:
crew_results=ranyx_crew.kickoff(
    inputs={
        "product_name":"coffee machine for the office",
        "websites_list":["www.amazon.eg","www.jumia.com.eg","www.noon.com/egypt-en"],
        "country_name":"Egypt",
        "no_keywords":10,
        "language":"Arabic",
        "score_th":0.10
    }
)

## Agent C:
To extract product details from any web page + HTML Parser code/tool

In [ ]:
from scrapegraph_py import Client
import json

def web_scraping_tool(page_url:str,required_fields:list):
    """ AN ai TOOL TAHT HELP A PERSON TO SCRAPE A WEB PAGE """   
    details=scrape_client.smartscraper(
        website_url=page_url,
        user_prompt="Extract "+json.dumps(required_fields,ensure_ascii=False)+" From the web page"
    )

    return {
        "page_url":page_url,
        "details":details
    }

scraping_agent=Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is desgined to help in looking for required values from any website url.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_task=Task(
    description="\n".join([
        "The Task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries",
        "ignore any susbicious links or not an ecommerce single product website link",
        "ignore any search results with confidence score less than ({score_th})",
        "The search product will be used to compare prices of product from different websites."
    ]),
    expected_outputs="A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir,"step_1_suggested_search_queries.json"),
    agent=search_engine_agent
)

In [ ]:
#page_details=web_scraping_tool(
 #   page_url="",
  #  required_fields=["product_title","current_price","old_price"]
#)

class ProductSpecs(BasedModel):
    specification_name:str
    specification_value:str

class SingleExtractedProduct(BaseModel):
    product_url:str=Field(...,title="The original url of the product page")
    product_title:str=Field(...,title="The title of the product")
    product_image_url:str=Field(...,title="The url of the product image")
    product_url:str=Field(...,"The url of the product")
    product_current_price:float=Field(...,"The current price of the product")
    product_original_price:float=Field(...,"The original price of the product before discount. Set to None if no discount.",default=None)
    product_discount_percentage:float=Field(...,"The discount percentage of the product. Set to None if no discount.",default=None)

    product_specs=List[ProductSpec]=Field(...,title="The specification of the product. Focus on the most important specs to compare.",min_items=1,max_items=5)
    agent_recommendation_rank:int=Field(...,title="The rank of the product (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst.")
    agent_recommendation_notes:List[str]=Field(...,title="Why would you recommnend or not reciommend this product.")

class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduuct]

@tool
def web_scraping_tool(page_url:str):
    details=scrape_client.smartscraper(
        website_url=website_url,
        user_promot="Extract '''json\n"+AllExtractedProducts.schema_json()+"'''\n the web page"
    )
    return{
        "page_url":page_url,
        "details":details
    }

scraping_agent=Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is desgined to help in looking for required values from any website url.",
    llm=basic_llm,
    verbose=True,
    tools=[web_scraping_tool]
)

scraping_task=Task(
    description="\n".join([
        "The Task is to extract the product from any ecommerce store page url.",
        "The task has to collect results from multiple pages urls.",
        "Collect the {top_recommendation_no} products from the search results."
    ]),
    expected_outputs="A JSON object containing the products details.",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir,"step_3_search_results.json"),
    agent=sscraping_agent
)



## AGENT D
To generate a professional procurement report + HTML generator code/tool 
It generates a visual reports

In [ ]:
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

procurement_report_author_agent=Agent(
    role="Procurement Report Author Agent",
    goal="To generate a professional HTMNL page for the procurement report",
    backstory="The agent is designed to assist in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm=basic_llm,
    verbose=True
)

procurement_report_author_task=Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the comapny to amke a specializied report."
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),
    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir,"step_4_procurement_report.html"),
    agent=procurement_report_author_agent
)

In [ ]:
about_company = "Rankyx is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

## run the crewai
rankyx_crew=Crew(
    agents=[
        search_queries_recommendation_agent,
        search_engine_agent,
        scraping_agent,
        procurement_report_author_agent
        
    ],
    tasks=[
        search_queries_recommendation_task,
        search_engine_task,
        scraping_task,
        procurement_report_author_task
    ],
    process=Process.sequential,
    knowledge_sources=[comapny_context]
)

In [ ]:
crew_results=rankyx_crew.kickoff(
    inputs={
        "product_name":"coffee machine for the office",
        "websites_list":["www.amazon.eg","www.jumia.com.eg","www.noon.com/egypt-en"],
        "country name":"Egypt",
        "no_keywords":10,
        "language":"English",
        "score_th":0.10,
        "top_recommendation_no":10
    }
)